In [38]:
import requests
import pandas as pd
import numpy as np
from io import StringIO

In [77]:
# Primero entenderme con los csv y ya cuando tal alomejor me pongo con json para traer los datos en inglés

url = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000013/~latest"
params = {"locale": "en"}
headers = {"Accept": "application/json"}

response = requests.get(url, params=params, headers=headers)
data = response.json()

### Variable Descriptions

Description of variable MOVIMIENTO_AERONAVE:

1. Otros servicios comerciales (Other commercial services)

This includes air operations that are commercial, but are not regular passenger or cargo transportation. Examples:

Sightseeing flights (aerial tours)

Crop spraying or aerial work

Aerial advertising (planes with banners)

Aerial photography

Air taxi (though sometimes it falls under “Non-regular”)

In other words, these are services that generate revenue but are not scheduled airline or traditional charter operations.

2. No regular (Non-regular)

Also called charter flights. These are commercial passenger or cargo flights that do not follow a fixed schedule or a regular frequency. Examples:

A flight hired by a group for an event

A humanitarian flight (if commercial)

On-demand operations

3. Regular (regular)

Flights that are scheduled commercial services, with established frequency, fixed routes, and open sale to the public. Examples:

Traditional airlines (LATAM, Iberia, etc.)

Daily/weekly flights with published timetables

4. Comercial (comercial)

This is a generic term that usually encompasses all flights that are for profit (i.e., passenger, cargo, or mail transportation for payment). However, in some datasets, it’s used as a base category when no further detail is provided. It may include both regular and non-regular flights.

In our case is the sum of Regular and Non-Regular

Ignore:
ESTADO_OBSERVACION, CONFIDENCIALIDAD_OBSERVACION

## Tráfico por islas y principales territorios de escala o de origen/destino

### 01 Tráfico de pasajeros (Passenger Traffic)

Tráfico de pasajeros registrados en los aeropuertos de las islas de Canarias según movimientos (llegadas y salidas), servicios comerciales y principales territorios de escala (territorio de despegue anterior al territorio de llegada independientemente del origen del vuelo / territorio de aterrizaje siguiente al territorio de salida independientemente del destino del vuelo). Datos mensuales y anuales para Canarias desde el año 2004.

Número de personas embarcadas y desembarcadas (entradas y salidas) de una aeronave en un aeropuerto, excluyendo a los miembros en servicio de las tripulaciones de vuelo y cabina.

For each island (TERRITORIO), for each destination/origin airport (AEROPUERTO_ESCALA), for each type of flight (MOVIMIENTO_AERONAVE), for each month provides the value of the passengers arrived from that airport (MOVIMIENTO_AERONAVE#es: Llegada), heading to that airport (MOVIMIENTO_AERONAVE#es: Salida) and Total \

In [23]:
# URL of the CSV API
url = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000013/~latest.csv?locale=en"

In [24]:
# Send HTTP GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Read CSV data using pandas
    csv_data = StringIO(response.content.decode("utf-8"))
    data = pd.read_csv(csv_data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

In [68]:
data.columns

Index(['MEDIDAS#es', 'MEDIDAS_CODE', 'TERRITORIO#es', 'TERRITORIO_CODE',
       'AEROPUERTO_ESCALA#es', 'AEROPUERTO_ESCALA_CODE',
       'MOVIMIENTO_AERONAVE#es', 'MOVIMIENTO_AERONAVE_CODE',
       'SERVICIO_AEREO#es', 'SERVICIO_AEREO_CODE', 'TIME_PERIOD#es',
       'TIME_PERIOD_CODE', 'OBS_VALUE', 'ESTADO_OBSERVACION#es',
       'ESTADO_OBSERVACION_CODE', 'CONFIDENCIALIDAD_OBSERVACION#es'],
      dtype='object')

In [31]:
df_passengers_only = data.copy(deep=True)

In [32]:
df_passengers_only.shape

(215040, 16)

In [33]:
df_passengers_only = df_passengers_only.drop(columns=df_passengers_only.columns[df_passengers_only.columns.str.endswith('_CODE')])

In [34]:
# Contains one value for every row (Pasajeros)
df_passengers_only.drop(['MEDIDAS#es', 'CONFIDENCIALIDAD_OBSERVACION#es', 'ESTADO_OBSERVACION#es'], axis=1, inplace=True)

In [35]:
df_passengers_only.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215040 entries, 0 to 215039
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   TERRITORIO#es           215040 non-null  object 
 1   AEROPUERTO_ESCALA#es    215040 non-null  object 
 2   MOVIMIENTO_AERONAVE#es  215040 non-null  object 
 3   SERVICIO_AEREO#es       215040 non-null  object 
 4   TIME_PERIOD#es          215040 non-null  object 
 5   OBS_VALUE               143567 non-null  float64
dtypes: float64(1), object(5)
memory usage: 9.8+ MB


In [36]:
for c in df_passengers_only.select_dtypes("object").columns:
    if c == "TIME_PERIOD#es":
        continue
    print(c)
    print(df_passengers_only[c].unique())

TERRITORIO#es
['Canarias' 'Lanzarote' 'Fuerteventura' 'Gran Canaria' 'Tenerife'
 'La Gomera' 'La Palma' 'El Hierro']
AEROPUERTO_ESCALA#es
['Reino Unido' 'Extranjero' 'Canarias'
 'Extranjeros y España (excluida Canarias)' 'España (excluida Canarias)'
 'España' 'Total' 'Alemania']
MOVIMIENTO_AERONAVE#es
['Llegada' 'Salida' 'Total']
SERVICIO_AEREO#es
['Comercial' 'Otros servicios comerciales' 'No regular' 'Regular']


In [37]:
df_passengers_only

,TERRITORIO#es,AEROPUERTO_ESCALA#es,MOVIMIENTO_AERONAVE#es,SERVICIO_AEREO#es,TIME_PERIOD#es,OBS_VALUE
0,Canarias,Reino Unido,Llegada,Comercial,01/2004,317378.0
1,Canarias,Reino Unido,Llegada,Comercial,02/2004,320960.0
2,Canarias,Reino Unido,Llegada,Comercial,06/2010,251362.0
3,Canarias,Reino Unido,Llegada,Comercial,07/2010,296367.0
4,Canarias,Reino Unido,Llegada,Comercial,08/2010,287040.0
...,...,...,...,...,...,...
215035,El Hierro,Alemania,Total,Regular,01/2010,NaN
215036,El Hierro,Alemania,Total,Regular,02/2010,NaN
215037,El Hierro,Alemania,Total,Regular,03/2010,NaN
215038,El Hierro,Alemania,Total,Regular,04/2010,NaN


Check if Comercial is the sum of all other three SERIVICIO_AEREO

In [45]:
# Pivot the data to have 'Comercial', 'Regular', 'No regular' as columns
pivot_df = df_passengers_only.pivot_table(
    index=['TERRITORIO#es', 'AEROPUERTO_ESCALA#es', 'MOVIMIENTO_AERONAVE#es', 'TIME_PERIOD#es'],
    columns='SERVICIO_AEREO#es',
    values='OBS_VALUE',
    aggfunc='sum'  # in case duplicates exist
).reset_index()

# List of required columns
required_services = ['Comercial', 'Regular', 'No regular']
for col in required_services:
    if col not in pivot_df.columns:
        pivot_df[col] = np.nan

# Compute expected Comercial = Regular + No regular
pivot_df['sum'] = pivot_df['Regular'].fillna(0) + pivot_df['No regular'].fillna(0) + pivot_df['Otros servicios comerciales'].fillna(0)
pivot_df['Comercial_actual'] = pivot_df['Comercial'].fillna(0)

# Check if they are equal (allowing for small floating point differences)
tolerance = 1e-6
pivot_df['Is_Valid'] = np.isclose(
    pivot_df['Comercial_actual'],
    pivot_df['sum'],
    atol=tolerance
)

# Select only relevant columns for review
validation_report = pivot_df[[
    'TERRITORIO#es',
    'AEROPUERTO_ESCALA#es',
    'MOVIMIENTO_AERONAVE#es',
    'TIME_PERIOD#es',
    'Comercial_actual',
    'Regular',
    'No regular',
    'sum',
    'Is_Valid'
]]

# Display results
print("Validation Report:")
print(validation_report)

# Optional: Show only invalid rows
invalid_rows = validation_report[~validation_report['Is_Valid']]
if len(invalid_rows) > 0:
    print("\n Invalid Entries (Comercial ≠ Regular + No regular):")
    print(invalid_rows)
else:
    print("\n All entries are valid.")

Validation Report:
SERVICIO_AEREO#es TERRITORIO#es AEROPUERTO_ESCALA#es MOVIMIENTO_AERONAVE#es  \
0                      Canarias             Alemania                Llegada   
1                      Canarias             Alemania                Llegada   
2                      Canarias             Alemania                Llegada   
3                      Canarias             Alemania                Llegada   
4                      Canarias             Alemania                Llegada   
...                         ...                  ...                    ...   
53755                  Tenerife                Total                  Total   
53756                  Tenerife                Total                  Total   
53757                  Tenerife                Total                  Total   
53758                  Tenerife                Total                  Total   
53759                  Tenerife                Total                  Total   

SERVICIO_AEREO#es TIME_PERIOD#es

In [ ]:
#df_passengers_only.loc[(df_passengers_only['TIME_PERIOD#es'] == "07/2025") & (df_passengers_only['SERVICIO_AEREO#es'] == "Comercial")].to_csv('borrar.csv', index=False)
#df_passengers_only.loc[(df_passengers_only['TERRITORIO#es'] == "Canarias") & (df_passengers_only['TIME_PERIOD#es'] == "07/2025") & (df_passengers_only["AEROPUERTO_ESCALA#es"] == "Reino Unido") & (df_passengers_only["MOVIMIENTO_AERONAVE#es"] == "Total")].to_csv('borrar.csv', index=False)

### 02 Tráfico de Mercancías y correos (Goods and Mail Traffic)

Tráfico de mercancías y correos registrados en los aeropuertos de las islas de Canarias según movimientos (llegadas y salidas), servicios comerciales y principales territorios de escala (territorio de despegue anterior al territorio de llegada independientemente del origen del vuelo / territorio de aterrizaje siguiente al territorio de salida independientemente del destino del vuelo). Datos mensuales y anuales para Canarias desde el año 2004.

Goods
Peso del total de mercancía transportada, expresado en kilogramos. Donde mercancía es cualquier propiedad transportada en una aeronave que no sea correo, suministros ni equipaje.
Mail
Peso del total de correo transportado, expresado en kilogramos. Donde correo es el envío de correspondencia y otros objetos transportados en una aeronave, que han sido enviados y destinados a ser entregados a las administraciones postales.

In [37]:
# URL of the CSV API
url = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000014/~latest.csv"

In [4]:
# Send HTTP GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Read CSV data using pandas
    csv_data = StringIO(response.content.decode("utf-8"))
    data = pd.read_csv(csv_data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

NameError: name 'url' is not defined

In [39]:
df_gm_only = data.copy(deep=True)

In [42]:
df_gm_only.drop(columns=df_gm_only.columns[df_gm_only.columns.str.endswith('_CODE')], inplace=True)

In [81]:
df_gm_only

,MEDIDAS#es,TERRITORIO#es,AEROPUERTO_ESCALA#es,MOVIMIENTO_AERONAVE#es,SERVICIO_AEREO#es,TIME_PERIOD#es,OBS_VALUE,ESTADO_OBSERVACION#es,CONFIDENCIALIDAD_OBSERVACION#es
0,Correo,Canarias,Reino Unido,Llegada,Comercial,01/2004,NaN,NaN,NaN
1,Correo,Canarias,Reino Unido,Llegada,Comercial,02/2004,NaN,NaN,NaN
2,Correo,Canarias,Reino Unido,Llegada,Comercial,03/2004,40.0,Valor normal,NaN
3,Correo,Canarias,Reino Unido,Llegada,Comercial,04/2004,NaN,NaN,NaN
4,Correo,Canarias,Reino Unido,Llegada,Comercial,05/2004,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
430075,Mercancía,El Hierro,Alemania,Total,Regular,2019,NaN,NaN,NaN
430076,Mercancía,El Hierro,Alemania,Total,Regular,01/2020,NaN,NaN,NaN
430077,Mercancía,El Hierro,Alemania,Total,Regular,02/2020,NaN,NaN,NaN
430078,Mercancía,El Hierro,Alemania,Total,Regular,03/2020,NaN,NaN,NaN


In [ ]:
#df_gm_only.to_csv("borrar.csv", index=False)

### 03 Tráfico de Operaciones

Tráfico de operaciones registradas en los aeropuertos de las islas de Canarias según movimientos (llegadas y salidas), servicios comerciales y principales territorios de origen/destino (territorio de origen o destino independientemente de si ha hecho o hará escala en otro territorio). Datos mensuales y anuales para Canarias desde el año 2004.

Operations
Número total de operaciones de aterrizaje y despegue efectuados por las aeronaves en el aeropuerto.

In [47]:
url = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000015/~latest.csv"

In [48]:
# Send HTTP GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Read CSV data using pandas
    csv_data = StringIO(response.content.decode("utf-8"))
    data = pd.read_csv(csv_data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

In [49]:
df_op_only = data.copy(deep=True)

In [50]:
df_op_only.drop(columns=df_op_only.columns[df_op_only.columns.str.endswith('_CODE')], inplace=True)

In [82]:
df_op_only

,MOVIMIENTO_AERONAVE#es,MEDIDAS#es,AEROPUERTO_ORIGEN_DESTINO#es,SERVICIO_AEREO#es,TIME_PERIOD#es,TERRITORIO#es,OBS_VALUE,ESTADO_OBSERVACION#es,CONFIDENCIALIDAD_OBSERVACION#es
0,Total,Operaciones,Reino Unido,Regular,01/2004,Canarias,425.0,Valor normal,NaN
1,Total,Operaciones,Reino Unido,Regular,02/2004,Canarias,378.0,Valor normal,NaN
2,Total,Operaciones,Reino Unido,Regular,03/2004,Canarias,358.0,Valor normal,NaN
3,Total,Operaciones,Reino Unido,Regular,04/2004,Canarias,280.0,Valor normal,NaN
4,Total,Operaciones,Reino Unido,Regular,05/2004,Canarias,187.0,Valor normal,NaN
...,...,...,...,...,...,...,...,...,...
215035,Llegada,Operaciones,Alemania,Comercial,04/2023,El Hierro,NaN,NaN,NaN
215036,Llegada,Operaciones,Alemania,Comercial,05/2023,El Hierro,NaN,NaN,NaN
215037,Llegada,Operaciones,Alemania,Comercial,06/2023,El Hierro,NaN,NaN,NaN
215038,Llegada,Operaciones,Alemania,Comercial,07/2023,El Hierro,NaN,NaN,NaN


In [ ]:
#df_op_only.to_csv("borrar.csv", index=False)

## Tráfico según tipos de servicios comerciales por islas y principales territorios de escala o de origen/destino

### 04 Tráfico de pasajeros, mercancías, correos y operaciones

Datasets 1, 2 and 3 put together in one

## Tráfico según tipos de servicios comerciales detallados por aeropuertos

### 05 Tráfico total de pasajeros, mercancías, correos y operaciones

Como el dataset 04 pero te pone cada aeropuerto de origen/destino y el total de MOVIMIENTO_AERONAVE, es decir llegadas y salidas



In [58]:
url_passengers = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000001/~latest.csv"
url_gm = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000002/~latest.csv"
url_op = "https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/C00017A_000003/~latest.csv"

In [60]:
# Send HTTP GET request
response = requests.get(url_passengers)

# Check if the request was successful
if response.status_code == 200:
    # Read CSV data using pandas
    csv_data = StringIO(response.content.decode("utf-8"))
    data_t = pd.read_csv(csv_data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

In [70]:
df_all = data_t.copy(deep=True)

In [71]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2963520 entries, 0 to 2963519
Data columns (total 16 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   SERVICIO_AEREO#es                object 
 1   SERVICIO_AEREO_CODE              object 
 2   MEDIDAS#es                       object 
 3   MEDIDAS_CODE                     object 
 4   MOVIMIENTO_AERONAVE#es           object 
 5   MOVIMIENTO_AERONAVE_CODE         object 
 6   TIME_PERIOD#es                   object 
 7   TIME_PERIOD_CODE                 object 
 8   AEROPUERTO_BASE#es               object 
 9   AEROPUERTO_BASE_CODE             object 
 10  AEROPUERTO_ESCALA#es             object 
 11  AEROPUERTO_ESCALA_CODE           object 
 12  OBS_VALUE                        float64
 13  ESTADO_OBSERVACION#es            object 
 14  ESTADO_OBSERVACION_CODE          object 
 15  CONFIDENCIALIDAD_OBSERVACION#es  float64
dtypes: float64(2), object(14)
memory usage: 361.8+ MB


In [72]:
df_all['AEROPUERTO_BASE_CODE']

0          ES_GCXO
1          ES_GCXO
2          ES_GCXO
3          ES_GCXO
4          ES_GCXO
            ...   
2963515    ES_GCRR
2963516    ES_GCRR
2963517    ES_GCRR
2963518    ES_GCRR
2963519    ES_GCRR
Name: AEROPUERTO_BASE_CODE, Length: 2963520, dtype: object

In [65]:
df_all.drop(columns=df_all.columns[df_all.columns.str.endswith('_CODE')], inplace=True)

In [66]:
# Movimiento aeronave is "Total" for all values
df_all.drop(['MOVIMIENTO_AERONAVE#es', 'CONFIDENCIALIDAD_OBSERVACION#es', 'MEDIDAS#es', 'ESTADO_OBSERVACION#es'], axis=1, inplace=True)

In [67]:
df_all

,SERVICIO_AEREO#es,TIME_PERIOD#es,AEROPUERTO_BASE#es,AEROPUERTO_ESCALA#es,OBS_VALUE
0,Comercial,02/2014,Aeropuerto de Tenerife Norte,Cabo Verde,2.0
1,Comercial,03/2014,Aeropuerto de Tenerife Norte,Cabo Verde,NaN
2,Comercial,04/2014,Aeropuerto de Tenerife Norte,Cabo Verde,NaN
3,Comercial,05/2014,Aeropuerto de Tenerife Norte,Cabo Verde,NaN
4,Comercial,06/2014,Aeropuerto de Tenerife Norte,Cabo Verde,NaN
...,...,...,...,...,...
2963515,Regular,10/2013,Aeropuerto de Lanzarote,Aeropuerto de Malmö Sturup,NaN
2963516,Regular,11/2013,Aeropuerto de Lanzarote,Aeropuerto de Malmö Sturup,NaN
2963517,Regular,12/2013,Aeropuerto de Lanzarote,Aeropuerto de Malmö Sturup,NaN
2963518,Regular,2013,Aeropuerto de Lanzarote,Aeropuerto de Malmö Sturup,NaN


We check again if SERVICIO_AEREO "Comercial" is equal to the sum of the other tree values in SERVICIO_AEREO

In [69]:
# Pivot the data to have 'Comercial', 'Regular', 'No regular' as columns
pivot_df = df_all.pivot_table(
    index=['AEROPUERTO_BASE#es', 'AEROPUERTO_ESCALA#es', 'TIME_PERIOD#es'],
    columns='SERVICIO_AEREO#es',
    values='OBS_VALUE',
    aggfunc='sum'  # in case duplicates exist
).reset_index()

# List of required columns
required_services = ['Comercial', 'Regular', 'No regular']
for col in required_services:
    if col not in pivot_df.columns:
        pivot_df[col] = np.nan

# Compute expected Comercial = Regular + No regular
pivot_df['sum'] = pivot_df['Regular'].fillna(0) + pivot_df['No regular'].fillna(0) + pivot_df['Otros servicios comerciales'].fillna(0)
pivot_df['Comercial_actual'] = pivot_df['Comercial'].fillna(0)

# Check if they are equal (allowing for small floating point differences)
tolerance = 1e-6
pivot_df['Is_Valid'] = np.isclose(
    pivot_df['Comercial_actual'],
    pivot_df['sum'],
    atol=tolerance
)

# Select only relevant columns for review
validation_report = pivot_df[[
    'AEROPUERTO_BASE#es',
    'AEROPUERTO_ESCALA#es',
    'TIME_PERIOD#es',
    'Comercial_actual',
    'Regular',
    'No regular',
    'sum',
    'Is_Valid'
]]

# Display results
print("Validation Report:")
print(validation_report)

# Optional: Show only invalid rows
invalid_rows = validation_report[~validation_report['Is_Valid']]
if len(invalid_rows) > 0:
    print("\n Invalid Entries (Comercial ≠ Regular + No regular):")
    print(invalid_rows)
else:
    print("\n All entries are valid.")

Validation Report:
SERVICIO_AEREO#es           AEROPUERTO_BASE#es  \
0                  Aeropuerto de Fuerteventura   
1                  Aeropuerto de Fuerteventura   
2                  Aeropuerto de Fuerteventura   
3                  Aeropuerto de Fuerteventura   
4                  Aeropuerto de Fuerteventura   
...                                        ...   
740875                                Canarias   
740876                                Canarias   
740877                                Canarias   
740878                                Canarias   
740879                                Canarias   

SERVICIO_AEREO#es                 AEROPUERTO_ESCALA#es TIME_PERIOD#es  \
0                  Aeropuerto Brussels South Charleroi        01/2004   
1                  Aeropuerto Brussels South Charleroi        01/2005   
2                  Aeropuerto Brussels South Charleroi        01/2006   
3                  Aeropuerto Brussels South Charleroi        01/2007   
4              

In [13]:
df_all.loc[(df_all['TIME_PERIOD#es'] == "06/2025") & (df_all["AEROPUERTO_BASE#es"] == "Aeropuerto de Lanzarote")].to_csv("borrar.csv", index=False)

### 06 Entrada de pasajeros, mercancías y correos y operaciones de aterrizaje

Como el dataset 05 pero solo entrada

### 07 Salida de pasajeros, mercancías y correos y operaciones de despegue

Como el dataset 05 pero solo salida